In [2]:
import pandas as pd
import numpy as np
import datetime
from EvGym import config

In [3]:
df_elaad = df_elaad = pd.read_csv("data/elaadnl_open_ev_datasets.csv", parse_dates = ["UTCTransactionStart", "UTCTransactionStop"])
df_elaad = df_elaad.rename(columns = config.elaad_rename)

In [4]:
df_elaad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   session               10000 non-null  int64         
 1   ChargePoint           10000 non-null  object        
 2   Connector             10000 non-null  int64         
 3   starttime_parking     10000 non-null  datetime64[ns]
 4   endtime_parking       10000 non-null  datetime64[ns]
 5   StartCard             10000 non-null  object        
 6   connected_time_float  10000 non-null  float64       
 7   charged_time_float    10000 non-null  float64       
 8   total_energy          10000 non-null  float64       
 9   max_power             10000 non-null  float64       
dtypes: datetime64[ns](2), float64(4), int64(2), object(2)
memory usage: 781.4+ KB


In [6]:
config.timestep

3600

In [7]:
df_elaad["start_hour"] = df_elaad["starttime_parking"].dt.hour
df_elaad["day_no"] = (pd.to_datetime(df_elaad["starttime_parking"].dt.date) - datetime.datetime(year = 2019, month = 1, day = 1)).dt.days

# Clean vehicles that have higher charged_time than connected time
df_elaad["charged_time_float"] = np.minimum(df_elaad["charged_time_float"], df_elaad["connected_time_float"])

df_elaad["energy_supplied"] = np.minimum(config.alpha_c * config.eta_c * df_elaad["charged_time_float"], config.B * config.FINAL_SOC)
df_elaad["soc_arr"] = (1 - df_elaad["energy_supplied"]/ (config.B))


df_elaad["charged_time"] = np.ceil(df_elaad["charged_time_float"]) 
df_elaad["connected_time"] = np.ceil(df_elaad["connected_time_float"])

starttime_min = pd.to_datetime("2000-01-01 00:00:00")
df_elaad["ts_arr"] = np.floor((df_elaad["starttime_parking"] - starttime_min).dt.total_seconds() / config.timestep)


if config.timestep == 3600:
    # This way it is consistent with connected_time
    df_elaad["ts_dep"] = df_elaad["ts_arr"] + df_elaad["connected_time"]
else:
    df_elaad["ts_dep"] = np.ceil((df_elaad["endtime_parking"] - starttime_min).dt.total_seconds() / config.timestep)
df_elaad["ts_soj"] = df_elaad["ts_dep"] - df_elaad["ts_arr"]
df_elaad["laxity"] = df_elaad["connected_time"] - (config.FINAL_SOC - df_elaad["soc_arr"]) * config.B / (config.alpha_c*config.eta_c)
df_elaad["depart_hour"] = df_elaad["start_hour"] + df_elaad["connected_time"] # Look how 
df_elaad["xi"] = df_elaad["laxity"] * config.psi * config.alpha_c

df_elaad = df_elaad.sort_values(by = ["ts_arr", "ts_dep"]).reset_index(drop = True)

In [8]:
df_elaad.describe()

,session,Connector,connected_time_float,charged_time_float,total_energy,max_power,start_hour,day_no,energy_supplied,soc_arr,charged_time,connected_time,ts_arr,ts_dep,ts_soj,laxity,depart_hour,xi
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,3.452817e+06,1.360300,5.822735,2.878022,13.635216,5.589366,12.412600,190.979600,28.608123,0.642398,3.355300,6.328300,171155.923000,171162.251300,6.328300,3.897120,18.740900,21.001189
std,1.076139e+05,0.480111,7.096361,2.644282,14.282036,3.333663,4.532131,110.531348,20.586961,0.257337,2.650125,7.093627,2652.843492,2653.132903,7.093627,6.334915,9.166023,34.138224
min,3.261657e+06,1.000000,0.020000,0.020000,0.100000,0.167000,0.000000,0.000000,0.215600,0.030000,1.000000,1.000000,166560.000000,166568.000000,1.000000,0.222635,2.000000,1.199755
25%,3.360979e+06,1.000000,1.510000,1.250000,4.530000,3.398000,9.000000,93.000000,13.475000,0.528375,2.000000,2.000000,168804.000000,168808.750000,2.000000,0.752635,13.000000,4.055872
50%,3.452182e+06,1.000000,3.380000,2.240000,7.830000,3.640000,12.000000,190.000000,24.147200,0.698160,3.000000,4.000000,171128.000000,171133.500000,4.000000,1.182635,16.000000,6.373099
75%,3.547667e+06,2.000000,8.410000,3.500000,18.262500,7.277750,16.000000,294.000000,37.730000,0.831562,4.000000,9.000000,173622.250000,173629.000000,9.000000,5.024119,21.000000,27.074475
max,3.634120e+06,2.000000,161.170000,25.670000,93.929000,22.500000,23.000000,364.000000,77.600000,0.997305,26.000000,162.000000,175317.000000,175336.000000,162.000000,155.024119,175.000000,835.409508


In [9]:
df_elaad[["connected_time", "ts_soj"]].describe()

,connected_time,ts_soj
count,10000.000000,10000.000000
mean,6.328300,6.328300
std,7.093627,7.093627
min,1.000000,1.000000
25%,2.000000,2.000000
50%,4.000000,4.000000
75%,9.000000,9.000000
max,162.000000,162.000000


In [10]:
df_elaad[["depart_hour"]].describe()

,depart_hour
count,10000.000000
mean,18.740900
std,9.166023
min,2.000000
25%,13.000000
50%,16.000000
75%,21.000000
max,175.000000


In [11]:
df_elaad.to_csv("data/df_elaad_preproc.csv", index = False)

In [12]:
df_elaad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   session               10000 non-null  int64         
 1   ChargePoint           10000 non-null  object        
 2   Connector             10000 non-null  int64         
 3   starttime_parking     10000 non-null  datetime64[ns]
 4   endtime_parking       10000 non-null  datetime64[ns]
 5   StartCard             10000 non-null  object        
 6   connected_time_float  10000 non-null  float64       
 7   charged_time_float    10000 non-null  float64       
 8   total_energy          10000 non-null  float64       
 9   max_power             10000 non-null  float64       
 10  start_hour            10000 non-null  int64         
 11  day_no                10000 non-null  int64         
 12  energy_supplied       10000 non-null  float64       
 13  soc_arr          

In [10]:
df_elaad.head()

,session,ChargePoint,Connector,starttime_parking,endtime_parking,StartCard,connected_time_float,charged_time_float,total_energy,max_power,...,energy_supplied,soc_arr,charged_time,connected_time,ts_arr,ts_dep,ts_soj,laxity,depart_hour,xi
0,3261657,e62c50d1be0a2f80ec51d471f9630a4e,2,2019-01-01 00:30:00,2019-01-01 08:24:00,826d337c1d84900452edfcb27a3fb11bcbd06f35da8406...,7.91,1.00,6.53,9.818,...,10.7800,0.865250,1.0,8.0,166560.0,166568.0,8.0,7.222635,8.0,38.922057
1,3262038,9bae10789a789973cc7f05d2a96df76f,1,2019-01-01 10:36:00,2019-01-01 12:52:00,26612ba143072faddaa884c2b04208992c5f44d97e5a24...,2.27,2.27,8.08,3.600,...,24.4706,0.694118,3.0,3.0,166570.0,166573.0,3.0,0.952635,13.0,5.133652
2,3262129,5ab468315a1f42feb6d0a87307593352,1,2019-01-01 11:49:00,2019-01-01 17:50:00,8478920deafe847713d1707d8f71a043c59fa066e55020...,6.02,3.75,11.56,3.400,...,40.4250,0.494688,4.0,7.0,166571.0,166578.0,7.0,3.472635,18.0,18.713681
3,3262170,00fa448dfcba79a12a3cea56fb48d63f,2,2019-01-01 12:22:00,2019-01-01 14:29:00,e8e1953d3e6221f971ce3299d2fa8275cdc9e7700e2b39...,2.12,0.87,3.84,7.973,...,9.3786,0.882768,1.0,3.0,166572.0,166575.0,3.0,2.352635,15.0,12.678113
4,3262208,e35645c6b4adb10224ecfbdff6bc5473,1,2019-01-01 12:51:00,2019-01-01 15:16:00,6490d53fbbe95d74592928292858f9b156d838f08d062e...,2.41,2.41,8.50,3.592,...,25.9798,0.675253,3.0,3.0,166572.0,166575.0,3.0,0.812635,15.0,4.379206
